# The detailed demonstration of the tropical rainfall diagnostic for ICON data:

# Calculation of histograms

### Content


1. [The load of packages, modules, and data](#1-load-of-packages-modules-and-data)

2. [Calculation of histogram of precipitation/tropical precipitation](#2-calculation-of-histogram-of-precipitationtropicalprecipitation)

    2.1. [with continuous  uniform binning](#21-with-continuous-uniform-binning)

    2.2  [with non-uniform (log-spaced) binning](#22-with-non-uniform-log-spaced-binning)

    2.3. [in the lazy (or delayed) mode](#23-in-the-lazy-or-delayed-mode)
    
    2.4. [with non-default tropical latitude band](#24-with-non-default-latitude-band)

    2.5  [with a specific time band](#25-with-a-specific-time-band)

    2.6. [with weights (weights=reader.grid_area)](#26-with-weights-weightsreadergrid_area)
 

3. [Loading the histogram to/from storage](#3-loading-the-histogram-tofrom-storage)

[Go to the end of file](#the-end)

#

## 1. The load of packages, modules, and data

In [1]:
import sys
from aqua import Reader
sys.path.insert(0, '../')
from tropical_rainfall_class import TR_PR_Diagnostic as TR_PR_Diag

FDB5 binary library not present on system, disabling FDB support.


#### ICON data

In [2]:
reader = Reader(model="ICON", exp="ngc2009", source="lra-r100-monthly")
icon = reader.retrieve(streaming=True, stream_step=1, stream_unit='days') 

## 2. Calculation of histogram of tropical precipitation


### 2.1. with continuous uniform binning 

##### We can perform calculations of the histograms with uniform binning by  initializing the following attributes of diagnostic: 
 - num_of_bins
 - first_edge
 - width_of_bin
 
 ##### In this case, all the bins in the histogram will have the width of bins.

In [3]:
diag = TR_PR_Diag(num_of_bins = 20, first_edge = 0, width_of_bin = 1*10**(-6)/20)

##### We calculate the histogram simply by function:

In [4]:
hist_icon_trop  = diag.histogram(icon)
hist_icon_trop

The obtained xarray.Dataset doesn't have global attributes. Consider adding global attributes manually to the dataset.


/work/bb1153/b382267/mamba_dir/mambaforge/envs/aqua/lib/python3.10/site-packages/dask/array/core.py:3471: UserWarning: Passing an object to dask.array.from_array which is already a Dask collection. This can lead to unexpected behavior.
  warnings.warn(


<xarray.Dataset>
Dimensions:        (center_of_bin: 20)
Coordinates:
  * center_of_bin  (center_of_bin) float64 2.5e-08 7.5e-08 ... 9.25e-07 9.75e-07
    width          (center_of_bin) float64 5e-08 5e-08 5e-08 ... 5e-08 5e-08
Data variables:
    counts         (center_of_bin) float64 3.822e+03 1.241e+03 ... 0.0 0.0
    frequency      (center_of_bin) float64 0.5308 0.1724 0.1443 ... 0.0 0.0 0.0
    pdf            (center_of_bin) float64 1.062e+07 3.447e+06 ... 0.0 0.0

##### The output of the histogram function is xarray.Dataset, which has two coordinates 
- center_of_bin:   the center of each bin
- width:           width of each bin
##### and contains three variables:
- counts:       the number of observations that fall into each bin
- frequency:    the number of cases in each bin, normalized by the total number of counts. The sum of the frequencies equals 1.
- pdf:          the number of cases in each bin, normalized by the total number of counts and width of each bin. 

##### local and global attributes. Local attributes contain the information about the time and space grid for which diagnostic performed the calculations:
- time_band:    the value of time of the first and last element in the dataset and the frequency of the time grid
- lat_band:     the maximum and minimum values of the tropical latitude band and the frequency of the latitude grid
- lon_band:     the maximum and minimum values of the longitude and the frequency of the longitude grid

##### Global attribute `history` contains the information about when the histogram was calculated and values of `time_band`, `lat_band`, and `lon_band`.

### 2.2.  with non-uniform (log-spaced) binning  

##### If you want to perform the calculation for non-uniform binning, use the `bins` attribute of the diagnostic instead of  `num_of_bins`, `first_edge`, and `width_of_bin`.  

In [6]:
bins = [1.00000000e-09, 1.63789371e-09, 2.68269580e-09, 4.39397056e-09,
       7.19685673e-09, 1.17876863e-08, 1.93069773e-08, 3.16227766e-08,
       5.17947468e-08, 8.48342898e-08, 1.38949549e-07, 2.27584593e-07,
       3.72759372e-07, 6.10540230e-07, 1.00000000e-06]

In [7]:
diag = TR_PR_Diag(bins = bins)

In [8]:
hist_icon_trop_logspaced  = diag.histogram(icon)
hist_icon_trop_logspaced

The obtained xarray.Dataset doesn't have global attributes. Consider adding global attributes manually to the dataset.


/work/bb1153/b382267/mamba_dir/mambaforge/envs/aqua/lib/python3.10/site-packages/dask/array/core.py:3471: UserWarning: Passing an object to dask.array.from_array which is already a Dask collection. This can lead to unexpected behavior.
  warnings.warn(


<xarray.Dataset>
Dimensions:        (center_of_bin: 14)
Coordinates:
  * center_of_bin  (center_of_bin) float64 1.319e-09 2.16e-09 ... 8.053e-07
    width          (center_of_bin) float64 6.379e-10 1.045e-09 ... 3.895e-07
Data variables:
    counts         (center_of_bin) float64 156.0 252.0 329.0 ... 249.0 5.0 0.0
    frequency      (center_of_bin) float64 0.02354 0.03803 ... 0.0007545 0.0
    pdf            (center_of_bin) float64 3.69e+07 3.64e+07 ... 3.173e+03 0.0

### 2.3. in the lazy (or delayed) mode

##### To perform calculations in the so-called lazy mode, use the flag `lazy` in the histogram function. 

In [9]:
diag = TR_PR_Diag(num_of_bins = 20, first_edge = 0, width_of_bin = 1*10**(-6)/20)

In [10]:
hist_icon_lazy  = diag.histogram(icon,  lazy=True) 
hist_icon_lazy

The obtained xarray.Dataset doesn't have global attributes. Consider adding global attributes manually to the dataset.


/work/bb1153/b382267/mamba_dir/mambaforge/envs/aqua/lib/python3.10/site-packages/dask/array/core.py:3471: UserWarning: Passing an object to dask.array.from_array which is already a Dask collection. This can lead to unexpected behavior.
  warnings.warn(


<xarray.Dataset>
Dimensions:        (center_of_bin: 20)
Coordinates:
  * center_of_bin  (center_of_bin) float64 2.5e-08 7.5e-08 ... 9.25e-07 9.75e-07
    width          (center_of_bin) float64 5e-08 5e-08 5e-08 ... 5e-08 5e-08
Data variables:
    counts         (center_of_bin) float64 dask.array<chunksize=(20,), meta=np.ndarray>

##### `Note`: In the case of lazy calculation, the function's output will be different:  the xarray will contain only non-computed counts. If you want to add frequency and pdf variables to the histogram Dataset, apply the following function `histogram_to_xarray` (but only when you are actually ready to compute the histogram).

##### The function `data_with_global_atributes` argument is needed to populate Dataset with global attributes. 

In [12]:
diag.populate_histogram(tprate_dataset=hist_icon_lazy, data_with_global_atributes=icon)

<xarray.Dataset>
Dimensions:        (center_of_bin: 20)
Coordinates:
  * center_of_bin  (center_of_bin) float64 2.5e-08 7.5e-08 ... 9.25e-07 9.75e-07
    width          (center_of_bin) float64 5e-08 5e-08 5e-08 ... 5e-08 5e-08
Data variables:
    counts         (center_of_bin) float64 dask.array<chunksize=(20,), meta=np.ndarray>
    frequency      (center_of_bin) float64 dask.array<chunksize=(20,), meta=np.ndarray>
    pdf            (center_of_bin) float64 dask.array<chunksize=(20,), meta=np.ndarray>

### 2.4. with non-default latitude band 

##### The tropical latitude band, by default, is the following

In [7]:
diag.trop_lat

10

##### You can modify the tropical latitude band as 

In [9]:
diag.trop_lat = 20
diag.trop_lat

20

##### You can also modify the latitude band with the use of an argument `trop_lat` of the histogram function 

In [13]:
hist_icon_trop  = diag.histogram(icon, trop_lat=30)
hist_icon_trop

/work/bb1153/b382267/mamba_dir/mambaforge/envs/aqua/lib/python3.10/site-packages/dask/array/core.py:3471: UserWarning: Passing an object to dask.array.from_array which is already a Dask collection. This can lead to unexpected behavior.
  warnings.warn(


The obtained xarray.Dataset doesn't have global attributes. Consider adding global attributes manually to the dataset.


<xarray.Dataset>
Dimensions:        (center_of_bin: 20)
Coordinates:
  * center_of_bin  (center_of_bin) float64 2.5e-08 7.5e-08 ... 9.25e-07 9.75e-07
    width          (center_of_bin) float64 5e-08 5e-08 5e-08 ... 5e-08 5e-08
Data variables:
    counts         (center_of_bin) float64 1.454e+04 3.314e+03 ... 0.0 0.0
    frequency      (center_of_bin) float64 0.6733 0.1534 0.09523 ... 0.0 0.0 0.0
    pdf            (center_of_bin) float64 1.347e+07 3.069e+06 ... 0.0 0.0

### 2.5. with a specific time band 

##### You can specify `s_time` and `f_time` as integers. For example, 

In [14]:
diag = TR_PR_Diag( trop_lat = 30,  num_of_bins = 20, first_edge = 0, width_of_bin = 1*10**(-6)/15, s_time = 0, f_time = 47)

In [15]:
hist_icon = diag.histogram(icon)
hist_icon

The obtained xarray.Dataset doesn't have global attributes. Consider adding global attributes manually to the dataset.


/work/bb1153/b382267/mamba_dir/mambaforge/envs/aqua/lib/python3.10/site-packages/dask/array/core.py:3471: UserWarning: Passing an object to dask.array.from_array which is already a Dask collection. This can lead to unexpected behavior.
  warnings.warn(


<xarray.Dataset>
Dimensions:        (center_of_bin: 20)
Coordinates:
  * center_of_bin  (center_of_bin) float64 3.333e-08 1e-07 ... 1.233e-06 1.3e-06
    width          (center_of_bin) float64 6.667e-08 6.667e-08 ... 6.667e-08
Data variables:
    counts         (center_of_bin) float64 4.273e+03 1.505e+03 993.0 ... 0.0 0.0
    frequency      (center_of_bin) float64 0.5935 0.209 0.1379 ... 0.0 0.0 0.0
    pdf            (center_of_bin) float64 8.902e+06 3.135e+06 ... 0.0 0.0

##### Also, you can specify `s_time` and `f_time` as strings. For example, 

In [16]:
diag.s_time = '2020:01'
diag.f_time ='2020/03/20/12'

##### or

In [17]:
diag.s_time = '2020'
diag.f_time ='2020.03.20'

##### There is the possibility of specifying only the year band or only the months' band. For example, we can select June, July, and August in a whole dataset as

In [18]:
diag.s_month = 3
diag.f_month = 6 

### 2.6. with weights (weights=reader.grid_area)

##### The function provides the opportunity to calculate the histogram with weights. Compared to standard methods, such computations are `high-speed` because they are based on `dask_histogram` package.

In [16]:
diag = TR_PR_Diag(num_of_bins = 20, first_edge = 0, width_of_bin = 1*10**(-6)/15, s_time = 0, f_time = 47)

In [17]:
hist_icon_trop_weighted  = diag.histogram(icon, weights=reader.grid_area)
hist_icon_trop_weighted

The obtained xarray.Dataset doesn't have global attributes. Consider adding global attributes manually to the dataset.


/work/bb1153/b382267/mamba_dir/mambaforge/envs/aqua/lib/python3.10/site-packages/dask/array/core.py:3471: UserWarning: Passing an object to dask.array.from_array which is already a Dask collection. This can lead to unexpected behavior.
  warnings.warn(
/work/bb1153/b382267/mamba_dir/mambaforge/envs/aqua/lib/python3.10/site-packages/dask/array/core.py:3471: UserWarning: Passing an object to dask.array.from_array which is already a Dask collection. This can lead to unexpected behavior.
  warnings.warn(


<xarray.Dataset>
Dimensions:        (center_of_bin: 20)
Coordinates:
  * center_of_bin  (center_of_bin) float64 3.333e-08 1e-07 ... 1.233e-06 1.3e-06
    width          (center_of_bin) float64 6.667e-08 6.667e-08 ... 6.667e-08
Data variables:
    counts         (center_of_bin) float64 5.256e+13 1.853e+13 ... 0.0 0.0
    frequency      (center_of_bin) float64 0.5934 0.2092 0.1379 ... 0.0 0.0 0.0
    pdf            (center_of_bin) float64 8.901e+06 3.138e+06 ... 0.0 0.0

##### You can also parse only the `bins` attribute of the class  instead of the specification of `num_of_bins`, `first_edge`, and `width_of_bin`. 

##### But such calculations would be much `slower`! (at around 50 times)

In [18]:
diag = TR_PR_Diag(bins = bins)

In [19]:
hist_icon_trop_weighted  = diag.histogram(icon,  weights=reader.grid_area)
hist_icon_trop_weighted

The obtained xarray.Dataset doesn't have global attributes. Consider adding global attributes manually to the dataset.


/work/bb1153/b382267/mamba_dir/mambaforge/envs/aqua/lib/python3.10/site-packages/dask/array/core.py:3471: UserWarning: Passing an object to dask.array.from_array which is already a Dask collection. This can lead to unexpected behavior.
  warnings.warn(
/work/bb1153/b382267/mamba_dir/mambaforge/envs/aqua/lib/python3.10/site-packages/dask/array/core.py:3471: UserWarning: Passing an object to dask.array.from_array which is already a Dask collection. This can lead to unexpected behavior.
  warnings.warn(


<xarray.Dataset>
Dimensions:        (center_of_bin: 14)
Coordinates:
  * center_of_bin  (center_of_bin) float64 1.319e-09 2.16e-09 ... 8.053e-07
    width          (center_of_bin) float64 6.379e-10 1.045e-09 ... 3.895e-07
Data variables:
    counts         (center_of_bin) float64 1.918e+12 3.099e+12 ... 6.182e+10 0.0
    frequency      (center_of_bin) float64 0.02352 0.03801 ... 0.0007581 0.0
    pdf            (center_of_bin) float64 3.687e+07 3.638e+07 ... 3.188e+03 0.0

#

## 3. Loading the histogram to/from storage

#### Saving the histogram in storage. 

##### `Notice`: the name of the file will be unique and depends on the time band 

In [20]:
path_to_histogram='/work/bb1153/b382267/AQUA/histograms/'
diag.save_histogram(dataset=hist_icon, path_to_histogram=path_to_histogram, name_of_file='icon')

'/work/bb1153/b382267/AQUA/histograms/icon_2020-02-01T00_histogram.pkl'

#### Loading/opening the histogram from the storage 

##### The information about how to open and merge the list of histograms, see the notebook  `diagnostic_vs_streaming.ipynb`

In [21]:
diag.load_histogram(path_to_histogram='/work/bb1153/b382267/AQUA/histograms/icon_2020-01-20T00_2020-01-20T23_histogram.pkl')

<xarray.Dataset>
Dimensions:        (center_of_bin: 20)
Coordinates:
  * center_of_bin  (center_of_bin) float64 3.333e-08 1e-07 ... 1.233e-06 1.3e-06
    width          (center_of_bin) float64 6.667e-08 6.667e-08 ... 6.667e-08
Data variables:
    counts         (center_of_bin) float64 2.914e+05 1.456e+04 ... 182.0 141.0
    frequency      (center_of_bin) float64 0.8634 0.04316 ... 0.0004178
    pdf            (center_of_bin) float64 1.295e+07 6.474e+05 ... 6.267e+03
Attributes:
    CDI:                  Climate Data Interface version 1.8.3rc (http://mpim...
    Conventions:          CF-1.6
    comment:              Sapphire Dyamond (k203123) on l40687 (Linux 4.18.0-...
    grid_file_uri:        http://icon-downloads.mpimet.mpg.de/grids/public/mp...
    history:              ./icon at 20220512 1522142023-06-19 19:03:19 regrid...
    institution:          Max Planck Institute for Meteorology/Deutscher Wett...
    number_of_grid_used:  15
    references:           see MPIM/DWD publications
    source:               git@gitlab.dkrz.de:icon/icon-aes.git@87a1eaded69e87...
    title:                ICON simulation
    uuidOfHGrid:          0f1e7d66-637e-11e8-913b-51232bb4d8f9

[Back to the top of file](#structure)

#

##### The end